In [1]:
import torch
import pickle

with open( "x_train_hist.p", "rb" ) as file:
    x_train_hist = pickle.load(file)
    
with open( "y_train_hist.p", "rb" ) as file:
    y_train_hist = pickle.load(file)
    
with open( "x_valid_hist.p", "rb" ) as file:
    x_valid_hist = pickle.load(file)
    
with open( "y_valid_hist.p", "rb" ) as file:
    y_valid_hist = pickle.load(file)

In [2]:
import torch
import random
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

from mriqa_dataset import MRIQADataset
from networks import ClassicCNN, PhilsClassicCnn, CatNet, CustomResNet

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Link: https://arxiv.org/abs/2003.04696



In [3]:
# set random seeds for reproducibility
random.seed(21062020)
np.random.seed(21062020)
torch.manual_seed(21062020)
torch.cuda.manual_seed(21062020)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(torch.cuda.current_device())
torch.cuda.set_device(0)

net = CustomResNet(num_classes=5)
net = net.cuda()

optimizer = optim.SGD(net.parameters(), lr=1e-3)
ce = CrossEntropyLoss().cuda()

num_epochs = len(x_train_hist)
num_mini_batches = 13

loss_csv = open('losses.csv', 'w')
loss_csv.write('epoch,training,validation\n')

best_val_loss = 999
print("start training")
for epoch in range(num_epochs):
    epoch_loss = 0.0
    net.train()
    
    train_batches = x_train_hist[epoch]
    train_labels = y_train_hist[epoch]

    # train loop
    for sample, label in tqdm(zip(train_batches, train_labels), total=len(train_batches), leave=False):       
        sample = sample.cuda()
        label = label.cuda()

        prediction = net(sample)
        loss = ce(prediction, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print('[{}] train-loss: {}'.format(epoch, epoch_loss / num_mini_batches))
    loss_csv.write(str(epoch) + ',' + str(epoch_loss / num_mini_batches))
    loss_csv.flush()

    # validation loop
    net.eval()
    mean_validation_loss = 0
    num_validation_mini_batches = 11

    if epoch % 3 == 0:
        valid_batches = x_valid_hist[epoch//5]
        valid_labels = y_valid_hist[epoch//5]
        
        with torch.no_grad():
            for sample, label in tqdm(zip(valid_batches, valid_labels), total=len(valid_batches), leave=False):              
                sample = sample.cuda()
                label = label.cuda()

                prediction = net(sample)
                validation_loss = ce(prediction, label)

                mean_validation_loss += validation_loss.item()
                
        print('[{}] validation-loss: {}'.format(epoch, mean_validation_loss / num_validation_mini_batches))
        loss_csv.write(',' + str(mean_validation_loss / num_validation_mini_batches) + '\n')
        loss_csv.flush()

    # save best model
    if mean_validation_loss <= best_val_loss:
        torch.save({'epoch': epoch,
                    'model_state_dict': net.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': validation_loss.item()}, 'checkpoint_best_resnet')
        best_val_loss = mean_validation_loss

print('DONE.')

0
start training


[0] train-loss: 0.5162584965045636


[0] validation-loss: 0.43978590315038507


[1] train-loss: 0.4385003034885113


[2] train-loss: 0.432023837016179


...

[306] validation-loss: 0.6875350150195035


[307] train-loss: 0.28599586624365586


[308] train-loss: 0.6391289417560284


[309] train-loss: 0.44680827397566575


[309] validation-loss: 0.7705133828249845
DONE.


In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [5]:
prediction

tensor([[-1.2149e+02,  5.6693e+01, -1.5790e+02,  1.2513e+02,  9.7539e+01],
        [-4.9917e+01, -1.1710e+01, -1.3494e+00,  5.9592e+01,  3.7572e+00],
        [-7.8551e+00,  1.5121e+01, -2.3986e+01,  5.4095e+00,  1.1265e+01],
        [ 2.0342e+01, -1.6284e+01,  1.6081e+01, -5.8594e+00, -1.4447e+01],
        [ 3.4142e+00, -9.1235e-01,  2.1889e+00, -4.1047e+00, -6.6818e-01],
        [ 2.0658e+01, -1.1394e+01,  1.0917e+01, -4.7126e+00, -1.5542e+01],
        [-9.9185e+01,  6.3831e+01, -1.5177e+02,  9.7483e+01,  8.9504e+01],
        [-2.1625e+02,  1.9040e+02, -4.5689e+02,  2.2397e+02,  2.5720e+02],
        [ 1.9817e+01,  1.1169e+00, -1.7972e+00, -5.1613e+00, -1.4060e+01],
        [ 2.9316e+00, -2.7252e-01,  4.8315e-01, -2.4243e+00, -7.8901e-01],
        [ 1.2003e+01, -8.4898e+00,  9.5590e+00, -6.5386e+00, -6.7070e+00],
        [ 1.1108e+01, -8.6098e+00,  8.9377e+00, -4.6214e+00, -6.9799e+00],
        [-3.8377e+01,  2.4375e+01, -5.0766e+01,  2.1482e+01,  4.2994e+01],
        [ 1.9014e+01, -1.

In [6]:
label

tensor([3, 3, 1, 2, 2, 0, 3, 1, 1, 4, 2, 2, 4, 2, 0, 1, 2, 3, 3, 1, 3],
       device='cuda:0')

In [7]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_train_hist, y_train_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [8]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.38      0.99      0.55      6339
           1       0.90      0.45      0.60      6175
           2       0.36      0.00      0.01      6165
           3       0.99      0.89      0.94      6208
           4       0.75      0.72      0.74      6113

    accuracy                           0.61     31000
   macro avg       0.68      0.61      0.57     31000
weighted avg       0.67      0.61      0.57     31000



In [9]:
confusion_matrix(all_labels, all_predictions)

array([[6279,   25,    0,    0,   35],
       [2637, 2806,    4,   20,  708],
       [6123,    2,   23,    0,   17],
       [   0,    4,    0, 5527,  677],
       [1317,  298,   37,   59, 4402]], dtype=int64)

In [10]:
confusion_matrix(all_labels, all_predictions)

array([[6279,   25,    0,    0,   35],
       [2637, 2806,    4,   20,  708],
       [6123,    2,   23,    0,   17],
       [   0,    4,    0, 5527,  677],
       [1317,  298,   37,   59, 4402]], dtype=int64)

In [11]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [12]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.38      0.99      0.55      2727
           1       0.90      0.47      0.62      2714
           2       0.44      0.00      0.01      2710
           3       0.98      0.88      0.93      2714
           4       0.75      0.75      0.75      2735

    accuracy                           0.62     13600
   macro avg       0.69      0.62      0.57     13600
weighted avg       0.69      0.62      0.57     13600



In [13]:
print(confusion_matrix(all_labels, all_predictions))

[[2697   15    0    0   15]
 [1112 1270    5   14  313]
 [2690    0    7    0   13]
 [   0    0    0 2377  337]
 [ 511  125    4   31 2064]]


In [14]:
torch.save({'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': validation_loss.item()}, 'final_resnet')

In [22]:
model = torch.load("checkpoint_best_resnet")

In [16]:
model = CustomResNet(num_classes=5)
checkpoint = torch.load("checkpoint_best_default")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
model.cuda()

CustomResNet(
  (pre): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [17]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = model(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [18]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.72      0.27      0.40      2727
           1       0.00      0.00      0.00      2714
           2       0.00      0.00      0.00      2710
           3       0.57      1.00      0.73      2714
           4       0.14      0.40      0.21      2735

    accuracy                           0.34     13600
   macro avg       0.29      0.34      0.27     13600
weighted avg       0.29      0.34      0.27     13600



C:\Users\psodm\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
print(confusion_matrix(all_labels, all_predictions))

[[ 743    0    0   37 1947]
 [ 166    0    0  325 2223]
 [ 123    0    0   46 2541]
 [   0    0    0 2713    1]
 [   0    0    0 1628 1107]]
